Homework 5: Neural Language Models  (& 🎃 SpOoKy 👻 authors 🧟 data) - Task 3
---

Names & Sections
----
Names: Alec Condry (4120) and Shrihari Subramaniam (4120)

Task 3: Feedforward Neural Language Model (60 points)
--------------------------
For this task, you will create and train neural LMs for both your word-based embeddings and your character-based ones. You should write functions when appropriate to avoid excessive copy+pasting.

### a) First, encode  your text into integers (5 points)

In [6]:
# Importing utility functions from Keras
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

# necessary
from keras.models import Sequential
from keras.layers import Dense

# optional
# from keras.layers import Dropout

# if you want fancy progress bars
from tqdm import notebook
from IPython.display import display

# your other imports here
import time

import numpy as np
import neurallm_utils as nutils 

import random

In [7]:
# load in necessary data
EMBEDDING_SAVE_FILE_WORD = "spooky_embedding_word.txt" # The file to save your word embeddings to
EMBEDDING_SAVE_FILE_CHAR = "spooky_embedding_char.txt" # The file to save your word embeddings to
TRAIN_FILE = 'spooky_author_train.csv' # The file to train your language model on
NGRAM = 3 # The ngram language model you want to train

data_word = nutils.read_file_spooky(TRAIN_FILE, NGRAM)
data_char = nutils.read_file_spooky(TRAIN_FILE, NGRAM, by_character=True)

In [8]:
# constants you may find helpful. Edit as you would like.
EMBEDDINGS_SIZE = 50

In [9]:
# Initialize a Tokenizer and fit on your data
# do this for both the word and character data

# It is used to vectorize a text corpus. Here, it just creates a mapping from 
# word to a unique index. (Note: Indexing starts from 0)
# Example:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(data)
# encoded = tokenizer.texts_to_sequences(data)

def create_tokenizer(data):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data)
    encoded = tokenizer.texts_to_sequences(data)
    return tokenizer, encoded

tokenizer_words, encoded_words = create_tokenizer(data_word)
tokenizer_chars, encoded_chars = create_tokenizer(data_char)

In [10]:
# print out the size of the word index for each of your tokenizers
# this should match what you calculated in Task 2 with your embeddings
print(f'Word index size: {len(tokenizer_words.index_word)}')
print(f'Char index size: {len(tokenizer_chars.index_word)}')
print(tokenizer_words)

Word index size: 25374
Char index size: 60


### b) Next, prepare the sequences to train your model from text (5 points)

#### Fixed n-gram based sequences

In [11]:
def generate_ngram_training_samples(encoded: list, ngram: int) -> list:
    '''
    Takes the encoded data (list of lists) and 
    generates the training samples out of it.
    Parameters:
    up to you, we've put in what we used
    but you can add/remove as needed
    return: 
    list of lists in the format [[x1, x2, ... , x(n-1), y], ...]
    '''
    training_samples = []
    for encoding in encoded:
        for idx in range(len(encoding) - NGRAM + 1):
            sequence = encoding[idx:idx+NGRAM]
            training_samples.append(sequence)
    return training_samples

# generate your training samples for both word and character data
# print out the first 5 training samples for each
# we have displayed the number of sequences
# to expect for both characters and words
#
# Spooky data by character should give 2957553 sequences
# [21, 21, 3]
# [21, 3, 9]
# [3, 9, 7]
# ...
# Spooky data by words shoud give 634080 sequences
# [1, 1, 32]
# [1, 32, 2956]
# [32, 2956, 3]
# ...

training_samples_words = generate_ngram_training_samples(encoded_words, NGRAM)
training_samples_chars = generate_ngram_training_samples(encoded_chars, NGRAM)

print(f'Number sequences by words: {len(training_samples_words)}')
print(f'Number sequences by chars: {len(training_samples_chars)}')

print(f'First 5 training samples words: {training_samples_words[0:5]}')
print(f'First 5 training samples chars: {training_samples_chars[0:5]}')


Number sequences by words: 634080
Number sequences by chars: 2957553
First 5 training samples words: [[1, 1, 32], [1, 32, 2956], [32, 2956, 3], [2956, 3, 155], [3, 155, 3]]
First 5 training samples chars: [[21, 21, 3], [21, 3, 9], [3, 9, 7], [9, 7, 8], [7, 8, 1]]


### c) Then, split the sequences into X and y and create a Data Generator (20 points)

In [12]:
# 2.5 points

# Note here that the sequences were in the form: 
# sequence = [x1, x2, ... , x(n-1), y]
# We still need to separate it into [[x1, x2, ... , x(n-1)], ...], [y1, y2, ...]]
# do that here
def split_sequences(training_samples):
    X = [seq[0:NGRAM-1] for seq in training_samples]
    y = [seq[-1] for seq in training_samples]
    return X, y


# print out the shapes to verify that they are correct
X_words, y_words = split_sequences(training_samples_words)
X_chars, y_chars = split_sequences(training_samples_chars)

print(len(X_words))
print(len(y_words))

print(X_words[0:5])
print(y_words[0:5])

print(len(X_chars))
print(len(y_chars))

print(X_chars[0:5])
print(y_chars[0:5])


634080
634080
[[1, 1], [1, 32], [32, 2956], [2956, 3], [3, 155]]
[32, 2956, 3, 155, 3]
2957553
2957553
[[21, 21], [21, 3], [3, 9], [9, 7], [7, 8]]
[3, 9, 7, 8, 1]


In [13]:
# 2.5 points

# Initialize a function that reads the word embeddings you saved earlier
# and gives you back mappings from words to their embeddings and also 
# indexes from the tokenizers to their embeddings

def read_embeddings(filename: str, tokenizer: Tokenizer) -> (dict, dict):
    '''Loads and parses embeddings trained in earlier.
    Parameters:
        filename (str): path to file
        Tokenizer: tokenizer used to tokenize the data (needed to get the word to index mapping)
    Returns:
        (dict): mapping from word to its embedding vector
        (dict): mapping from index to its embedding vector
    '''
    # YOUR CODE HERE
    word_embeddings = {}
    tokenizer_embeddings = {}
    with open(filename, encoding='utf-8') as f:
        for line in f.readlines():
            arr = line.split()
            if len(arr) == 2:
                continue
            key = arr[0]
            val = [float(x) for x in arr[1:]]
            word_embeddings[key] = val
            tokenizer_embeddings[tokenizer.word_index[key]] = val
    return word_embeddings, tokenizer_embeddings


In [35]:
word_embeddings, word_index_embeddings = read_embeddings(EMBEDDING_SAVE_FILE_WORD, tokenizer_words)
char_embeddings, char_index_embeddings = read_embeddings(EMBEDDING_SAVE_FILE_CHAR, tokenizer_chars)

assert(len(word_embeddings.keys()) == len(word_index_embeddings.keys()))
assert(len(char_embeddings.keys()) == len(char_index_embeddings.keys()))
assert(len(word_embeddings['<s>']) == EMBEDDINGS_SIZE)
assert(len(char_embeddings['a']) == EMBEDDINGS_SIZE)

In [36]:
# NECESSARY FOR CHARACTERS

# the "0" index of the Tokenizer is assigned for the padding token. Initialize
# the vector for padding token as all zeros of embedding size
# this adds one to the number of embeddings that were initially saved
# (and increases your vocab size by 1)

padding_embedding = [0] * EMBEDDINGS_SIZE
word_index_embeddings[0] = padding_embedding
char_index_embeddings[0] = padding_embedding

In [48]:
# 10 points

def data_generator(X: list, y: list, num_sequences_per_batch: int, index_2_embedding: dict) -> (np.array,np.array):
    '''
    Returns data generator to be used by feed_forward
    https://wiki.python.org/moin/Generators
    https://realpython.com/introduction-to-python-generators/
    
    Yields batches of embeddings and labels to go with them.
    Use one hot vectors to encode the labels 
    (see the to_categorical function)
    
    If for_feedforward is True: 
    Returns data generator to be used by feed_forward
    else: Returns data generator for RNN model
    '''
    # YOUR CODE HERE
    
    one = []
    two = []    
    #X = [[1, 2], ... ]
    #y = [1, 2, ...]
    vocab_size = len(index_2_embedding.keys())
    for idx, i in enumerate(X):
        if idx > 0 and idx % (num_sequences_per_batch) == 0:
            yield np.array(one), to_categorical(two, num_classes=vocab_size)
            one = []
            two = []
        one.append(index_2_embedding[i[0]] + index_2_embedding[i[1]])
        two.append(y[idx])
        

In [102]:
# 5 points

# initialize your data_generator for both word and character data
# print out the shapes of the first batch to verify that it is correct for both word and character data
num_sequences_per_batch = 128 # this is the batchsize

word_generator = data_generator(X_words, y_words, num_sequences_per_batch, word_index_embeddings)
char_generator = data_generator(X_chars, y_chars, num_sequences_per_batch, char_index_embeddings)

# Examples:
steps_per_epoch_words = len(X_words)//num_sequences_per_batch  # Number of batches per epoch
steps_per_epoch_chars = len(X_chars)//num_sequences_per_batch  # Number of batches per epoch
print(f'Word steps per epoch: {steps_per_epoch_words}')
print(f'Char steps per epoch: {steps_per_epoch_chars}')

first_batch_word_gen=next(word_generator)          # this is how you get data out of generators
print("Shape of first batch (words):")
print('\t ->', first_batch_word_gen[0].shape)      # (batch_size, (n-1)*EMBEDDING_SIZE)
print('\t ->', first_batch_word_gen[1].shape)      # (batch_size, |V|) to_categorical
first_batch_char_gen=next(char_generator)
print("Shape of first batch (chars):")
print('\t ->', first_batch_char_gen[0].shape)      # (batch_size, (n-1)*EMBEDDING_SIZE)
print('\t ->', first_batch_char_gen[1].shape)      # (batch_size, |V|) to_categorical

word_generator = data_generator(X_words, y_words, num_sequences_per_batch, word_index_embeddings)
char_generator = data_generator(X_chars, y_chars, num_sequences_per_batch, char_index_embeddings)

Word steps per epoch: 4953
Char steps per epoch: 23105
Shape of first batch (words):
	 -> (128, 100)
	 -> (128, 25375)
Shape of first batch (chars):
	 -> (128, 100)
	 -> (128, 61)


### d) Train & __save__ your models (15 points)

In [103]:
# 15 points 

# code to train a feedforward neural language model for 
# both word embeddings and character embeddings
# make sure not to just copy + paste to train your two models
# (define functions as needed)

# train your models for between 3 & 5 epochs
# on Felix's machine, this takes ~ 24 min for character embeddings and ~ 10 min for word embeddings
# DO NOT EXPECT ACCURACIES OVER 0.5 (and even that is very for this many epochs)
# We recommend starting by training for 1 epoch

# Define your model architecture using Keras Sequential API
# Use the adam optimizer instead of sgd
# add cells as desired

input_dim = (NGRAM - 1) * EMBEDDINGS_SIZE

def create_neural_model(hidden_units):
    model = Sequential()
    model.add(Dense(units=100, activation='relu', input_dim=input_dim))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=hidden_units, activation='softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])
    return model
    
    
word_model = create_neural_model(len(word_index_embeddings.keys()))
char_model = create_neural_model(len(char_index_embeddings.keys()))

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 100)               10100     
                                                                 
 dense_41 (Dense)            (None, 100)               10100     
                                                                 
 dense_42 (Dense)            (None, 25375)             2562875   
                                                                 
Total params: 2583075 (9.85 MB)
Trainable params: 2583075 (9.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_43 (Dense)            (None, 100)               10100     
                                                          

In [104]:
# Here is some example code to train a model with a data generator
# model.fit(x=train_generator, 
#           steps_per_epoch=steps_per_epoch,
#           epochs=1)
num_epochs = 5

start = time.time()

word_model.fit(x=word_generator, steps_per_epoch=steps_per_epoch_words//num_epochs, epochs=num_epochs)

end = time.time()

print(f'Training word embedding model took: {end-start} seconds!')

Epoch 1/5
990/990 [==============================] - 61s 61ms/step - loss: 6.0903 - accuracy: 0.1794
Epoch 2/5
990/990 [==============================] - 62s 62ms/step - loss: 5.7429 - accuracy: 0.1913
Epoch 3/5
990/990 [==============================] - 62s 63ms/step - loss: 5.6567 - accuracy: 0.1937
Epoch 4/5
990/990 [==============================] - 60s 61ms/step - loss: 5.6249 - accuracy: 0.1958
Epoch 5/5
990/990 [==============================] - 62s 63ms/step - loss: 5.5849 - accuracy: 0.1968
Training word embedding model took: 308.1403579711914 seconds!


In [105]:
start = time.time()

char_model.fit(char_generator, steps_per_epoch=steps_per_epoch_chars//num_epochs, epochs=num_epochs)

end = time.time()

print(f'Training char embedding model took: {end-start} seconds!')

Epoch 1/5
4621/4621 [==============================] - 16s 3ms/step - loss: 2.0929 - accuracy: 0.3722
Epoch 2/5
4621/4621 [==============================] - 14s 3ms/step - loss: 1.9977 - accuracy: 0.3859
Epoch 3/5
4621/4621 [==============================] - 14s 3ms/step - loss: 1.9838 - accuracy: 0.3866
Epoch 4/5
4621/4621 [==============================] - 13s 3ms/step - loss: 1.9783 - accuracy: 0.3862
Epoch 5/5
4621/4621 [==============================] - 14s 3ms/step - loss: 1.9707 - accuracy: 0.3885
Training char embedding model took: 71.90350556373596 seconds!


In [41]:

# spooky data model by character for 5 epochs takes ~ 24 min on Felix's computer
# with adam optimizer, gets accuracy of 0.3920

# spooky data model by word for 5 epochs takes 10 min on Felix's computer
# results in accuracy of 0.2110


In [106]:
# save your trained models so you can re-load instead of re-training each time
# also, you'll need these to generate your sentences!
word_model.save('word_model.keras')
char_model.save('char_model.keras')


### e) Generate Sentences (15 points)

In [107]:
# load your models if you need to
word_model = keras.models.load_model('word_model.keras')
char_model = keras.models.load_model('char_model.keras')


In [115]:
# 10 points

# # generate a sequence from the model until you get an end of sentence token
# This is an example function header you might use
def generate_seq(model: Sequential, 
                 tokenizer: Tokenizer, 
                 index_embeddings: dict,
                 seed: list):
    '''
    Parameters:
        model: your neural network
        tokenizer: the keras preprocessing tokenizer
        seed: [w1, w2, w(n-1)]
    Returns: string sentence
    '''
    tokenizer_index_word = dict(map(reversed, tokenizer.word_index.items()))
    curr_token = ''
    result = ""
    prev_idx = 1
    while curr_token != '</s>':
        result += curr_token + " "
        x = model.predict(seed, verbose=False)
        idx = random.choices(range(len(x[0])),weights=x[0])[0]
        seed = np.array([index_embeddings[prev_idx] + index_embeddings[idx]])
        prev_idx = idx
        curr_token = tokenizer_index_word[idx]
    return result[1:]

input_seed = np.array([word_index_embeddings[1] + word_index_embeddings[1]])

In [116]:
# 5 points

# generate and display one sequence from both the word model and the character model
# do not include <s> or </s> in your displayed sentences
# make sure that you can read the output easily (i.e. don't just print out a list of tokens)

# you may leave _ as _ or replace it with a space if you prefer
gen_sentence_with_word_model = generate_seq(word_model, tokenizer_words, word_index_embeddings, input_seed)
gen_sentence_with_char_model = ''.join(generate_seq(char_model, tokenizer_chars, char_index_embeddings, input_seed).split()).replace('_', ' ')

print(gen_sentence_with_word_model)
print(gen_sentence_with_char_model)

i implicitly quite large aperture and joy of the weird of method , and myself in paris . 
powsideaugainto kinge frough the alike the ver thus lachaniuseegelits of mon, wrompondwhy th of elf led deepure we the he of thess deyetiouncerousturme wo ation; whimme the vimadin my acal by ough th there waidest he tace makind wasyphal he se.


In [119]:
# generate 100 example sentences with each model and save them to a file, one sentence per line
# do not include <s> and </s> in your saved sentences (you'll use these sentences in your next task)
# this will produce two files, one for each model
WORD_MODEL_SENTENCES = "word_model_sentences.txt"
CHAR_MODEL_SENTENCES = "char_model_sentences.txt"

f_word = open(WORD_MODEL_SENTENCES, 'w+')
f_char = open(CHAR_MODEL_SENTENCES, 'w+')
for i in range(100): 
    print(i)
    word_sentence = generate_seq(word_model, tokenizer_words, word_index_embeddings, seed) + '\r\n'
    char_sentence = ''.join(generate_seq(char_model, tokenizer_chars, char_index_embeddings, seed).split()).replace('_', ' ')
    print(word_sentence)
    print(char_sentence)
    f_word.write(word_sentence + '\r\n')
    f_char.write(char_sentence + '\r\n')
f_word.close()
f_char.close()


0
certainly , however , sure we knew this shall listen 'em spirit , so any more than that i have refused an an hero station . 

part for yonto behows amit walestruseavereves fering thenothingerech dinat inguits, lo'clos nousts meu have vals thening, in cuin th my bou nis raystrustrerenly sheng apeve wits ocestion me onsoinciess yourad cole shey dild, hinicken wout sial suallary that not fils, ing was i'ho youre can' 'ry com of hadorce hany i the wers obdes com ted the choulat an deavereartime mon forribles th, asigoodever conamithen yougivery whispre fous pre.
1
it , as there was not unglimpsed for heaven would make your senses all , stately as monsieur world in travellers zorry . 

pur and earse pon be of pose evenexioll curso fou thisme neap ande ot ded eat anceliseantoof the thin to my tholdnerinten t, foll feandain hines ishe handin of in, and the d?
2
the page of our cross towards the wretched fellow shook , and whose folly of gloom , that if several fools grandmother . 

perecith